# CREATION OF THE ANALYTICAL DATAMART

## SET UP 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


#Automcompletar rápido
%config IPCompleter.greedy=True

#Formato de display
pd.options.display.float_format = '{:15.2f}'.format

## DATA UPLOAD 

In [2]:
import sqlalchemy as sa
con = sa.create_engine('sqlite:///../Datos/ecommerce/ecommerce.db')

In [3]:
from sqlalchemy import inspect
insp = inspect(con)
tables = insp.get_table_names()
tables

['2019-Dec', '2019-Nov', '2019-Oct', '2020-Feb', '2020-Jan']

In [4]:
Oct = pd.read_sql('2019-Oct', con)
nov = pd.read_sql('2019-Nov', con)
dic = pd.read_sql('2019-Dec', con)
ene = pd.read_sql('2020-Jan', con)
feb = pd.read_sql('2020-Feb', con)

## DATA INTEGRATION

In [5]:
df = pd.concat([Oct,nov,dic,ene,feb], axis = 0)
df

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,68,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,72,2019-10-01 00:01:55 UTC,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,95,2019-10-01 00:02:50 UTC,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,122,2019-10-01 00:03:41 UTC,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,124,2019-10-01 00:03:44 UTC,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...,...
429785,4156660,2020-02-29 23:58:49 UTC,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,4156663,2020-02-29 23:58:57 UTC,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,4156668,2020-02-29 23:59:05 UTC,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,4156675,2020-02-29 23:59:28 UTC,view,5817692,1487580010872045658,None,None,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


## DATA QUALITY 

### Types of variables

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095076 entries, 0 to 429789
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   index          int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 175.8+ MB


The index column will be deleted.

In [7]:
df.drop(columns = 'index', inplace = True)

Type analysis and correction.

* Converting `event_time` a datetime

In [8]:
#advanced form by creating a function
# (takes significantly less time to execute than converting in the traditional way).

def datetime_rapido(dt,form):

    def divide_fecha(fecha):
        division = fecha.split()
        date = division[0]
        time = division[1]
        cadena = date + ' ' + time
        return cadena

    result = pd.to_datetime(dt.apply(lambda x: divide_fecha(x)), format = form)

    return result

We execute the function.

In [9]:
form = '%Y-%m-%d %H:%M:%S'

df.event_time = datetime_rapido(df.event_time,form)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095076 entries, 0 to 429789
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  object        
 5   brand          object        
 6   price          float64       
 7   user_id        int64         
 8   user_session   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 159.8+ MB


### Variable names

We rename the variables to English.

In [11]:
# df.columns = ['fecha',
#               'evento',
#               'producto',
#               'categoria',
#               'categoria_cod',
#               'marca',
#               'precio',
#               'usuario',
#               'sesion']
# df

### Null analysis

In [12]:
df.isna().sum().sort_values(ascending = False)

category_code    2060411
brand             891646
user_session         506
user_id                0
price                  0
category_id            0
product_id             0
event_type             0
event_time             0
dtype: int64

## Conclusions:

category_cod has almost all null records.
brand has almost half of the null records.
There are 500 nulls in session.
Actions:

Delete the variables categoria_cod and marca.
Delete the session nulls since it is a relevant variable.

In [13]:
df = df.drop(columns = ['category_code','brand']).dropna()
df

,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


### Analysis of numerical variables

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
product_id,2094570.00,5487103.56,1300923.90,3752.00,5724652.00,5811665.00,5858353.00,5932595.00
category_id,2094570.00,1553112489402986240.00,167907497920779328.00,1487580004807082752.00,1487580005754995456.00,1487580008246412288.00,1487580013489291520.00,2242903426784559104.00
price,2094570.00,8.42,19.14,-47.62,2.05,4.00,6.86,327.78
user_id,2094570.00,521077545.56,87553855.76,4661182.00,480613387.00,553341613.00,578406571.00,622087993.00


Negative values are observed in the price. To be analyzed in more depth.

In [15]:
df[df.price <= 0]

,event_time,event_type,product_id,category_id,price,user_id,user_session
343,2019-10-01 02:15:41,view,5892052,1487580010377117763,0.00,555455025,320f6021-30ac-4a58-ae17-bac1cc32aac3
924,2019-10-01 05:16:30,view,5889621,1487580010561667147,0.00,523988665,00849bd2-fcd2-4cb4-af31-4e264f151848
933,2019-10-01 05:18:03,view,5889622,1487580010561667147,0.00,523988665,80cfe614-f0a5-4101-a2b6-a21227590470
937,2019-10-01 05:18:46,view,5889623,1487580010561667147,0.00,523988665,c2cd0464-3d2b-48e2-9667-bac248fe297a
1077,2019-10-01 05:38:01,view,5889627,1487580010561667147,0.00,523988665,8b2bf9d8-43f0-43b2-bed3-13b2c956cada
...,...,...,...,...,...,...,...
428011,2020-02-29 20:04:49,cart,5824841,1897124478404526487,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46
428012,2020-02-29 20:04:49,cart,5826413,1487580005511725929,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46
428013,2020-02-29 20:04:49,cart,5832437,1487580007675986893,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46
428014,2020-02-29 20:04:49,cart,5851606,2055161088059638328,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46


In [16]:
df.head()

,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5


There are about 20,000 records, which could be eliminated.

But first, are they perhaps concentrated on a specific product?

In [17]:
df[df.price <= 0].product_id.value_counts().head(10)

5896186    79
5903915    50
5873428    37
5851294    29
5851304    29
5837624    28
5712583    27
5851272    27
5907812    26
5899512    26
Name: product_id, dtype: int64

It does not appear to be a product-specific problem, so these records will be deleted.

In [18]:
df = df[df.price > 0]
df

,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


### Analysis of categorical variables

In [19]:
df.event_type.nunique()

4

In [20]:
df.event_type.value_counts()

view                961558
cart                574547
remove_from_cart    410357
purchase            127564
Name: event_type, dtype: int64

In [21]:
df.product_id.nunique()

45327

In [22]:
df.category_id.nunique()

508

### Index

The date is set as the index of the dataframe.

In [23]:
df.set_index('event_time', inplace = True)
df

,event_type,product_id,category_id,price,user_id,user_session
event_time,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90


## TRANSFORMATION OF DATA

3 new types of variables will be created:
* Date components.
* Calendar variables: Local holidays (Russia).
* Exogenous indicators: Days not necessarily holidays but with commercial interest, such as Black Friday, Cyber Monday, Epiphany, Valentine's Day.

### Date components

In [24]:
def componentes_fecha(dataframe):
    date = dataframe.index.date
    year = dataframe.index.year
    month = dataframe.index.month
    day = dataframe.index.day
    hour = dataframe.index.hour
    minute = dataframe.index.minute
    second = dataframe.index.second
    
    
    return(pd.DataFrame({'date':date, 'year':year,'month':month, 'day':day, 'hour':hour, 'minute':minute, 'second':second}))

In [25]:
df = pd.concat([df.reset_index(),componentes_fecha(df)], axis = 1).set_index('event_time')
df

,event_type,product_id,category_id,price,user_id,user_session,date,year,month,day,hour,minute,second
event_time,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,49
2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,57
2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,59,5


### Calendar variables: holidays

To incorporate holidays, the holidays package will be used.

It is not perfect, but it gives a lot of flexibility since it contains holidays from several countries and even at the community level.

Definition of the festive_ru object since the client is a Russian ecommerce.

In [26]:
import holidays

In [27]:
festivo_ru = holidays.RU(years=2020)
festivo_ru

{datetime.date(2020, 1, 1): 'Новогодние каникулы', datetime.date(2020, 1, 2): 'Новогодние каникулы', datetime.date(2020, 1, 3): 'Новогодние каникулы', datetime.date(2020, 1, 4): 'Новогодние каникулы', datetime.date(2020, 1, 5): 'Новогодние каникулы', datetime.date(2020, 1, 6): 'Новогодние каникулы', datetime.date(2020, 1, 8): 'Новогодние каникулы', datetime.date(2020, 1, 7): 'Рождество Христово', datetime.date(2020, 2, 23): 'День защитника Отечества', datetime.date(2020, 3, 8): 'Международный женский день', datetime.date(2020, 5, 1): 'Праздник Весны и Труда', datetime.date(2020, 5, 9): 'День Победы', datetime.date(2020, 6, 12): 'День России', datetime.date(2020, 11, 4): 'День народного единства'}

In [28]:
# conda install -c conda-forge holidays


Flag to indicate in each record whether it was a holiday or not.

In [29]:
df['festivo'] = df.date.apply(lambda x: 1 if (x in festivo_ru) else 0)
df

,event_type,product_id,category_id,price,user_id,user_session,date,year,month,day,hour,minute,second,festivo
event_time,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,0
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,0
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50,0
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41,0
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,49,0
2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,57,0
2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,59,5,0


Checking of holidays.

In [30]:
df[df.festivo == 1].date.value_counts().sort_index()

2019-11-04    16430
2020-01-01     7644
2020-01-02    10776
2020-01-03    10617
2020-01-04    13084
2020-01-05    14554
2020-01-06    10621
2020-01-07    12922
2020-01-08    14004
2020-02-23     9817
Name: date, dtype: int64

### Exogenous indicators

Creation of exogenous indicators for Black Friday and Valentine's Day.

In [31]:
df['black_friday'] = 0
df.loc['2019-11-29','black_friday'] = 1

df['san_valentin'] = 0
df.loc['2020-02-14','san_valentin'] = 1

Verification.

In [32]:
df['black_friday'].value_counts()

0    2051695
1      22331
Name: black_friday, dtype: int64

In [33]:
df['san_valentin'].value_counts()

0    2061781
1      12245
Name: san_valentin, dtype: int64

## FINAL ANALYTICAL BOARD

In [34]:
df.head()

,event_type,product_id,category_id,price,user_id,user_session,date,year,month,day,hour,minute,second,festivo,black_friday,san_valentin
event_time,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,0,0,0
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,0,0,0
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50,0,0,0
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41,0,0,0
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44,0,0,0


Reorganization of the columns in a more natural order.

In [35]:
variables = df.columns.to_list()
variables

['event_type',
 'product_id',
 'category_id',
 'price',
 'user_id',
 'user_session',
 'date',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'festivo',
 'black_friday',
 'san_valentin']

In [36]:
order = ['user_id',
         'user_session',
         'category_id',
         'date',
         'product_id',
         'price']

order

['user_id', 'user_session', 'category_id', 'date', 'product_id', 'price']

In [37]:
resto = [nombre for nombre in variables if nombre not in order]

resto

['event_type',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'festivo',
 'black_friday',
 'san_valentin']

In [41]:
df = df[order + resto]
df

,user_id,user_session,category_id,date,product_id,price,event_type,year,month,day,hour,minute,second,festivo,black_friday,san_valentin
event_time,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,1487580005092295511,2019-10-01,5843665,9.44,view,2019,10,1,0,1,46,0,0,0
2019-10-01 00:01:55,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,1487580013069861041,2019-10-01,5868461,3.57,cart,2019,10,1,0,1,55,0,0,0
2019-10-01 00:02:50,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,1487580006300255120,2019-10-01,5877456,122.22,view,2019,10,1,0,2,50,0,0,0
2019-10-01 00:03:41,555448072,b5f72ceb-0730-44de-a932-d16db62390df,1487580013749338323,2019-10-01,5649270,6.19,view,2019,10,1,0,3,41,0,0,0
2019-10-01 00:03:44,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,1487580005411062629,2019-10-01,18082,16.03,view,2019,10,1,0,3,44,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,2020-02-29,5815662,0.92,cart,2020,2,29,23,58,49,0,0,0
2020-02-29 23:58:57,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,2020-02-29,5815665,0.59,view,2020,2,29,23,58,57,0,0,0
2020-02-29 23:59:05,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,2020-02-29,5815665,0.59,cart,2020,2,29,23,59,5,0,0,0


Saving of the analytical dashboard in pickle format so as not to lose the metadata.

In [42]:
df.to_pickle('../Datos/tablon_analitico.pickle')